### Notebook to build a specials measure

A notebook to help build a specials measure.

- [Imports](#IMPORTS)
- [](#f1)

<a id='IMPORTS'></a>
### Imports
Import libraries required for analysis

In [16]:
#import libraries required for analysis
from ebmdatalab import bq
import os
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

#set to display all rows in data
#pd.set_option('display.max_rows', None)
pd.option_context('display.float_format', '{:,.2f}'.format)

# Set display options for full width
pd.set_option('display.max_colwidth', None)  # Show full column content
pd.set_option('display.width', None)        # Adjust display width to the notebook's width

<a id='#f1'></a>
### Explore availability restriction
AMP table contains a column 'avail_restrict'. This uses a code to detail availability of products.
The table 'availabilityrestriction' details the meaning behind these codes.

In [17]:
sql = '''
SELECT * FROM dmd.availabilityrestriction
'''
avail_rest_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_rest.csv'))
avail_rest_DF = avail_rest_DF.fillna('None')
avail_rest_DF

,cd,descr
0,1,None
1,2,Restricted Availability
2,3,Individual Patient Supply
3,4,Imported
4,5,Clinical Trial
5,6,Special
6,7,Extemp
7,9,Not available
8,10,For Patient Safety Consideration


In [18]:
sql = '''
SELECT avail_restrict, COUNT(*) AS row_count
FROM `ebmdatalab.dmd.amp`
GROUP BY avail_restrict;
'''
avail_rest_count_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_rest_count.csv'))

merged_DF = avail_rest_DF.merge(avail_rest_count_DF, 
                                how='left', 
                                left_on='cd', 
                                right_on='avail_restrict')
# Drop the 'avail_restrict' column
merged_DF = merged_DF.drop(columns=['avail_restrict'])

# Fill NaN values in the 'row_count' column with 0
merged_DF['row_count'] = merged_DF['row_count'].fillna(0)

merged_DF['row_count'] = merged_DF['row_count'].astype(int)

merged_DF

,cd,descr,row_count
0,1,None,89846
1,2,Restricted Availability,57
2,3,Individual Patient Supply,24
3,4,Imported,659
4,5,Clinical Trial,1
5,6,Special,7281
6,7,Extemp,0
7,9,Not available,61257
8,10,For Patient Safety Consideration,4


We can discount items where avail_restrict = 1 (no restriction) and 9 (not available). There are also zero items with avail_restrict = 7 (Extemp).

<a id='#f2'></a>
### Explore availability restriction column data
We can extract the rows with avail_restrict values not 1, 7, 9 to explore further what products they contain. We can then try to identify the significance of these products by looking for number of items and spend in the English Prescribing data.

In [19]:
sql = '''
SELECT id, nm, descr, bnf_code 
FROM `ebmdatalab.dmd.amp` 
WHERE avail_restrict = 2
'''
avail_rest_2_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_rest_2.csv'))
avail_rest_2_DF

,id,nm,descr,bnf_code
0,41729811000001105,Vyvgart 400mg/20ml concentrate for solution for infusion vials,Vyvgart 400mg/20ml concentrate for solution for infusion vials (Argenx UK Ltd),NaN
1,42388711000001103,Nexpovio 20mg tablets,Nexpovio 20mg tablets (Menarini Stemline UK Ltd),NaN
2,42527511000001100,Livmarli 9.5mg/ml oral solution,Livmarli 9.5mg/ml oral solution (Mirum Pharmaceuticals International BV),NaN
3,42702411000001107,Vyvgart 1000mg/5.6ml solution for injection vials,Vyvgart 1000mg/5.6ml solution for injection vials (Argenx UK Ltd),NaN
4,43792511000001100,Leqembi 200mg/2ml concentrate for solution for infusion vials,Leqembi 200mg/2ml concentrate for solution for infusion vials (Eisai Ltd),NaN
5,43792811000001102,Leqembi 500mg/5ml concentrate for solution for infusion vials,Leqembi 500mg/5ml concentrate for solution for infusion vials (Eisai Ltd),NaN
6,44145611000001106,Comirnaty KP.2 adults and adolescents from 12 years COVID-19 mRNA Vaccine 30micrograms/0.3ml dose dispersion for injection multidose vials,Comirnaty KP.2 adults and adolescents from 12 years COVID-19 mRNA Vaccine 30micrograms/0.3ml dose dispersion for injection multidose vials (Pfizer Ltd),NaN
7,42977611000001107,Clozapine 25mg tablets,Clozapine 25mg tablets (Viatris UK Healthcare Ltd),0402010C0AAAAAA
8,42977211000001105,Clozapine 100mg tablets,Clozapine 100mg tablets (Viatris UK Healthcare Ltd),0402010C0AAABAB
9,3830311000001101,Clozaril 25mg tablets,Clozaril 25mg tablets (Viatris UK Healthcare Ltd),0402010C0BBAAAA


In [28]:
sql = '''
SELECT bnf_code, 
       bnf_name, 
       SUM(items) AS items, 
       SUM(actual_cost) AS cost
FROM `ebmdatalab.hscic.normalised_prescribing`
WHERE bnf_code IN (
    SELECT DISTINCT bnf_code 
    FROM `ebmdatalab.dmd.amp` 
    WHERE avail_restrict = 2
)
GROUP BY bnf_code, bnf_name
ORDER BY items DESC
'''
avail_rest_2_usage_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_restrict_2_usage.csv'))

avail_rest_2_usage_DF

,bnf_code,bnf_name,items,cost
0,0402010C0AAABAB,Clozapine 100mg tablets,12588,504203.85949
1,090607000BBLJA0,Healthy Start Children's Vitamin drops,9901,28589.76976
2,0402010C0AAAAAA,Clozapine 25mg tablets,7506,58419.94802
3,0402010C0AAABAB,Clozapine_Tab 100mg,5808,326376.58000
4,0402010C0AAAAAA,Clozapine_Tab 25mg,3425,32652.56000
5,0402010C0BCABAB,Denzapine 100mg tablets,2833,144920.28822
6,090607000BBLPA0,Healthy Start Women's Vitamin tablets,2633,3863.20610
7,0402010C0BBABAB,Clozaril 100mg tablets,1967,60604.36986
8,0402010C0BBABAB,Clozaril_Tab 100mg,1840,90906.12000
9,1404000AMBBAAAA,Gardasil 9 vacc inj 0.5ml pre-filled syringes,1451,172242.16817


**Restricted availability** denotes medications which are only prescribed from specialist services - not relevent to "specials" measure

In [29]:
sql = '''
SELECT id, nm, descr, bnf_code 
FROM `ebmdatalab.dmd.amp` 
WHERE avail_restrict = 3
'''
avail_rest_3_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_rest_3.csv'))
avail_rest_3_DF

,id,nm,descr,bnf_code
0,15514711000001108,"Factor XI 1,000unit powder and solvent for solution for infusion vials","Factor XI 1,000unit powder and solvent for solution for infusion vials (Bio Products Laboratory Ltd)",NaN
1,35820711000001107,Crysvita 10mg/1ml solution for injection vials,Crysvita 10mg/1ml solution for injection vials (Kyowa Kirin Ltd),NaN
2,35821111000001100,Crysvita 20mg/1ml solution for injection vials,Crysvita 20mg/1ml solution for injection vials (Kyowa Kirin Ltd),NaN
3,35821511000001109,Crysvita 30mg/1ml solution for injection vials,Crysvita 30mg/1ml solution for injection vials (Kyowa Kirin Ltd),NaN
4,39782811000001104,Vistogard 10g oral granules sachets,Vistogard 10g oral granules sachets (Protherics Medicines Development Ltd),NaN
5,42561311000001109,Oxervate 20micrograms/ml eye drops 1ml unit dose,Oxervate 20micrograms/ml eye drops 1ml unit dose (Chapper Healthcare),NaN
6,43998011000001102,Yorvipath 168micrograms/0.56ml solution for injection pre-filled pens,Yorvipath 168micrograms/0.56ml solution for injection pre-filled pens (Ascendis Pharma UK Ltd),NaN
7,43998811000001108,Yorvipath 294micrograms/0.98ml solution for injection pre-filled pens,Yorvipath 294micrograms/0.98ml solution for injection pre-filled pens (Ascendis Pharma UK Ltd),NaN
8,43999211000001102,Yorvipath 420micrograms/1.4ml solution for injection pre-filled pens,Yorvipath 420micrograms/1.4ml solution for injection pre-filled pens (Ascendis Pharma UK Ltd),NaN
9,44013311000001103,Dasiglucagon 4mg/1ml solution for infusion vials,Dasiglucagon 4mg/1ml solution for infusion vials (WEP Clinical),NaN


In [30]:
sql = '''
SELECT bnf_code, 
       bnf_name, 
       SUM(items) AS items, 
       SUM(actual_cost) AS cost
FROM `ebmdatalab.hscic.normalised_prescribing`
WHERE bnf_code IN (
    SELECT DISTINCT bnf_code 
    FROM `ebmdatalab.dmd.amp` 
    WHERE avail_restrict = 3
)
GROUP BY bnf_code, bnf_name
ORDER BY items DESC
'''
avail_rest_3_usage_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_restrict_3_usage.csv'))

avail_rest_3_usage_DF

,bnf_code,bnf_name,items,cost
0,0401030E0BBAAAA,Sodium Amytal 60mg capsules,7481,673365.11672
1,0401030T0BCAAAC,Tuinal 100mg Pulvules,7313,696800.10164
2,0401030T0BCAAAC,Tuinal_Pulvule 100mg,7279,507441.91000
3,0401030E0BBAAAA,Sod Amytal_Pulvule 60mg,6063,848094.45000
4,0401030H0BBAAAB,Soneryl_Tab 100mg,2650,233409.60000
5,0401030H0BBAAAB,Soneryl 100mg tablets,1859,211325.73693
6,1311070S0BDAEAA,BioBag 50 2.5cm x 4cm dressing kit,1619,372638.81023
7,0401030T0BBABAB,Seconal Sodium 100mg capsules,1610,332084.50744
8,0401030T0BBABAB,Seconal Sod_Pulvule 100mg,1519,241424.19000
9,1311070S0BDADAA,BioBag 100 4cm x 5cm dressing kit,1395,353998.12394


**Individual Patient Supply** denotes medications which are only prescribed for a select number of patients

In [31]:
sql = '''
SELECT id, nm, descr, bnf_code 
FROM `ebmdatalab.dmd.amp` test_DF
WHERE avail_restrict = 4
'''
avail_rest_4_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_rest_4.csv'))
avail_rest_4_DF

,id,nm,descr,bnf_code
0,29175011000001104,Iperten 10mg tablets,Iperten 10mg tablets (Imported (France)),NaN
1,29193611000001104,Proctolog ointment,Proctolog ointment (Imported (Spain)),NaN
2,31584411000001103,Dronabinol 2.5mg capsules,Dronabinol 2.5mg capsules (Imported (United States)),NaN
3,31736011000001109,Lumitene 30mg capsules,Lumitene 30mg capsules (Imported (United States)),NaN
4,32641911000001105,Triapten Antiviral 2% cream,Triapten Antiviral 2% cream (Imported (Germany)),NaN
...,...,...,...,...
654,19390111000001109,Abnoba Viscum Q 200micrograms/1ml solution for injection ampoules,Abnoba Viscum Q 200micrograms/1ml solution for injection ampoules (Imported (Germany)),190203000BBAAA0
655,19390511000001100,Abnoba Viscum Q 20mg/1ml solution for injection ampoules,Abnoba Viscum Q 20mg/1ml solution for injection ampoules (Imported (Germany)),190203000BBAAA0
656,19390811000001102,Abnoba Viscum Q 20micrograms/1ml solution for injection ampoules,Abnoba Viscum Q 20micrograms/1ml solution for injection ampoules (Imported (Germany)),190203000BBAAA0
657,19391111000001103,Abnoba Viscum Q 2mg/1ml solution for injection ampoules,Abnoba Viscum Q 2mg/1ml solution for injection ampoules (Imported (Germany)),190203000BBAAA0


In [32]:
sql = '''
SELECT bnf_code, 
       bnf_name, 
       SUM(items) AS items, 
       SUM(actual_cost) AS cost
FROM `ebmdatalab.hscic.normalised_prescribing`
WHERE bnf_code IN (
    SELECT DISTINCT bnf_code 
    FROM `ebmdatalab.dmd.amp` 
    WHERE avail_restrict = 4
)
GROUP BY bnf_code, bnf_name
ORDER BY items DESC
'''
avail_rest_4_usage_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_restrict_4_usage.csv'))

with pd.option_context('display.max_rows', 100):
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(avail_rest_4_usage_DF.head(100))

,bnf_code,bnf_name,items,cost
0,0602010V0AABZBZ,Levothyroxine sodium 100microgram tablets,121398229,"194,216,666.86"
1,0602010V0AABXBX,Levothyroxine sodium 50microgram tablets,109805716,"169,907,058.25"
2,0602010V0AABWBW,Levothyroxine sodium 25microgram tablets,107289528,"245,342,049.35"
3,0602010V0AABZBZ,Levothyrox Sod_Tab 100mcg,31541510,"58,139,478.48"
4,0602010V0AABXBX,Levothyrox Sod_Tab 50mcg,28604796,"52,320,495.91"
5,0602010V0AABWBW,Levothyrox Sod_Tab 25mcg,27607357,"91,205,218.91"
6,1003020AAAAABAB,Capsaicin 0.025% cream,1600933,"30,953,424.15"
7,1003020AAAAAAAA,Capsaicin 0.075% cream,799709,"12,781,324.05"
8,0301020I0AAACAC,Ipratropium bromide 500micrograms/2ml neb liq ud vials,737865,"9,505,273.87"
9,1003020AAAAABAB,Capsaicin_Crm 0.025%,476869,"9,238,076.82"


**Imported** denotes imported medications. The 'descr' field details Imported and the country imported from. However the BNF code isn't unique to imported products - for example 0602010V0AABZBZ	Levothyroxine sodium 100microgram tablets - seems to include standard UK product and imported products.

In [33]:
sql = '''
SELECT id, nm, descr, bnf_code 
FROM `ebmdatalab.dmd.amp` 
WHERE avail_restrict = 5
'''
avail_rest_5_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_rest_5.csv'))
avail_rest_5_DF

,id,nm,descr,bnf_code
0,41823211000001105,COVID-19 Vaccine Moderna (mRNA-1283.222) 10micrograms/0.2ml dose solution for injection vials,"COVID-19 Vaccine Moderna (mRNA-1283.222) 10micrograms/0.2ml dose solution for injection vials (Moderna, Inc)",1404000ATBRADBJ


In [38]:
sql = '''
SELECT bnf_code, 
       bnf_name, 
       SUM(items) AS items, 
       SUM(actual_cost) AS cost
FROM `ebmdatalab.hscic.normalised_prescribing`
WHERE bnf_code IN (
    SELECT DISTINCT bnf_code 
    FROM `ebmdatalab.dmd.amp` 
    WHERE avail_restrict = 5
)
GROUP BY bnf_code, bnf_name
ORDER BY items DESC
'''
avail_rest_5_usage_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_restrict_5_usage.csv'))

avail_rest_5_usage_DF

,bnf_code,bnf_name,items,cost


**Clinical Trial** denotes a medication used for clinical trial. Only one product, no prescribing.

In [34]:
sql = '''
SELECT id, nm, descr, bnf_code 
FROM `ebmdatalab.dmd.amp` 
WHERE avail_restrict = 10
'''
avail_rest_10_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_rest_10.csv'))
avail_rest_10_DF

,id,nm,descr,bnf_code
0,26345611000001105,Phenol 90% cutaneous solution - should NOT be prescribed,Phenol 90% cutaneous solution - should NOT be prescribed (Special Order),NaN
1,26346211000001102,Phenol 95% cutaneous solution - should NOT be prescribed,Phenol 95% cutaneous solution - should NOT be prescribed (Special Order),NaN
2,29718111000001107,Phenol 80% cutaneous solution - should NOT be prescribed,Phenol 80% cutaneous solution - should NOT be prescribed (Special Order),NaN
3,22235811000001100,Phenol liquefied - should NOT be prescribed,Phenol liquefied - should NOT be prescribed (Special Order),1311050P0AAABAB


In [39]:
sql = '''
SELECT bnf_code, 
       bnf_name, 
       SUM(items) AS items, 
       SUM(actual_cost) AS cost
FROM `ebmdatalab.hscic.normalised_prescribing`
WHERE bnf_code IN (
    SELECT DISTINCT bnf_code 
    FROM `ebmdatalab.dmd.amp` 
    WHERE avail_restrict = 10
)
GROUP BY bnf_code, bnf_name
ORDER BY items DESC
'''
avail_rest_10_usage_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_restrict_4_usage.csv'))

avail_rest_10_usage_DF

,bnf_code,bnf_name,items,cost
0,1311050P0AAABAB,Phenol liquefied,78,4465.70855
1,1311050P0AAABAB,Phenol Liquefied_,69,2016.57000


**For Patient Safety Consideration** denotes medication which appear not to be recommended for prescribing (all denote "should NOT be prescribed").

In [40]:
sql = '''
SELECT id, nm, descr, bnf_code 
FROM `ebmdatalab.dmd.amp` 
WHERE avail_restrict = 6
'''
avail_rest_6_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_rest_6.csv'))
avail_rest_6_DF

Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


,id,nm,descr,bnf_code
0,19980211000001105,Potassium ascorbate 10% eye drops preservative free,Potassium ascorbate 10% eye drops preservative free (Special Order),None
1,20399911000001104,Coal tar 30% in Emulsifying ointment,Coal tar 30% in Emulsifying ointment (Special Order),None
2,25192611000001104,Iodine 1% in Boric acid ear powder,Iodine 1% in Boric acid ear powder (Special Order),None
3,25407511000001104,Hydrochloric acid 18% dental solution,Hydrochloric acid 18% dental solution (Special Order),None
4,25566311000001101,Fentanyl 24.75mg/990ml infusion bags,Fentanyl 24.75mg/990ml infusion bags (Special Order),None
...,...,...,...,...
7276,32284311000001104,Cade oil 12.5% in Emulsifying ointment,Cade oil 12.5% in Emulsifying ointment (Special Order),190605000AACECE
7277,24551811000001101,Ginger syrup,Ginger syrup (Special Order),190607000AAAGAG
7278,18159811000001109,Eosin 1% solution,Eosin 1% solution (Special Order),190800000AAAPAP
7279,17656311000001103,Eosin 2% solution,Eosin 2% solution (Drug Tariff Special Order),190800000AABTBT


In [43]:
sql = '''
SELECT bnf_code, 
       bnf_name, 
       SUM(items) AS items, 
       SUM(actual_cost) AS cost
FROM `ebmdatalab.hscic.normalised_prescribing`
WHERE bnf_code IN (
    SELECT DISTINCT bnf_code 
    FROM `ebmdatalab.dmd.amp` 
    WHERE avail_restrict = 6
)
GROUP BY bnf_code, bnf_name
ORDER BY items DESC
'''
avail_rest_6_usage_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_restrict_6_usage.csv'))

with pd.option_context('display.max_rows', 100):
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(avail_rest_6_usage_DF.head(100))

,bnf_code,bnf_name,items,cost
0,0410030C0AAAFAF,Methadone 1mg/ml oral solution sugar free,9126099,"61,590,579.30"
1,0410030C0AAAAAA,Methadone 1mg/ml oral solution,4742757,"31,371,963.65"
2,0410030C0AAAFAF,Methadone HCl_Mix 1mg/1ml S/F,2017025,"15,008,152.64"
3,0410030C0AAAAAA,Methadone HCl_Mix 1mg/1ml,1869833,"15,461,493.30"
4,0901020D0AAAHAH,Cyanocobalamin 1mg modified-release tablets,1216501,"7,558,454.02"
5,0906040G0AAELEL,Colecalciferol 400unit tablets,693619,"5,557,402.90"
6,0407010Q0AAAAAA,Co-Proxamol_Tab 32.5mg/325mg,551243,"15,051,015.47"
7,0401010ADAACZCZ,Melatonin 1mg/ml oral solution sugar free,484045,"70,526,833.96"
8,0407010Q0AAAAAA,Co-proxamol 32.5mg/325mg tablets,350255,"35,862,526.11"
9,0906040G0AACRCR,"Colecalciferol 20,000unit tablets",297544,"5,421,599.07"


<a id='#f3'></a>
### Focus on specials
Firstly look at how many specials are denoted by BNF code that only signifies a special.


In [50]:
sql = '''
SELECT 
    bnf_code,
    COUNT(*) AS count
FROM 
    `ebmdatalab.dmd.amp`
WHERE 
    avail_restrict = 6
GROUP BY 
    bnf_code
ORDER BY 
    count DESC;
'''
avail_rest_count_6_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_rest_count_6.csv'))

Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [51]:
sql = '''
SELECT 
    bnf_code,
    COUNT(*) AS count
FROM 
    `ebmdatalab.dmd.amp`
WHERE 
    avail_restrict != 6
GROUP BY 
    bnf_code
ORDER BY 
    count DESC;
'''
avail_rest_count_not_6_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','avail_rest_count_not_6.csv'))

Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [52]:
avail_rest_count_6_DF

,bnf_code,count
0,None,1404
1,190203000BBAAA0,46
2,1305020C0AAFPFP,6
3,1305020C0AAAVAV,4
4,1305020C0AABSBS,4
...,...,...
4433,190605000AACECE,1
4434,190607000AAAGAG,1
4435,190800000AAAPAP,1
4436,190800000AABTBT,1


In [56]:
merged_df = avail_rest_count_6_DF.merge(
    avail_rest_count_not_6_DF,
    on='bnf_code',
    how='left',
    suffixes=('_special', '_not_special')
)
merged_df = merged_df.dropna(subset=['bnf_code'])
merged_df

,bnf_code,count_special,count_not_special
1,190203000BBAAA0,46,397
2,1305020C0AAFPFP,6,2
3,1305020C0AAAVAV,4,<NA>
4,1305020C0AABSBS,4,<NA>
5,1305020S0AACZCZ,4,<NA>
...,...,...,...
4433,190605000AACECE,1,<NA>
4434,190607000AAAGAG,1,1
4435,190800000AAAPAP,1,<NA>
4436,190800000AABTBT,1,<NA>


In [57]:
sql = '''
SELECT DISTINCT
    bnf_code,
    nm
FROM 
    `ebmdatalab.dmd.vmp`
'''
vmp_DF = bq.cached_read(sql, csv_path=os.path.join('../','data','vmp_data.csv'))

In [58]:
merged_df = merged_df.merge(
    vmp_DF,
    on='bnf_code',
    how='left'
)
merged_df

,bnf_code,count_special,count_not_special,nm
0,190203000BBAAA0,46,397,Graphites 9% homeopathic cream
1,190203000BBAAA0,46,397,Ruta graveolens 9% homeopathic cream
2,190203000BBAAA0,46,397,Rhus toxicodendron 9% homeopathic cream
3,190203000BBAAA0,46,397,Tamus communis 9% homeopathic cream
4,190203000BBAAA0,46,397,Tea tree oil 4.78% homeopathic cream
...,...,...,...,...
6207,190605000AACECE,1,<NA>,Cade oil 12.5% in Emulsifying ointment
6208,190607000AAAGAG,1,1,Ginger syrup
6209,190800000AAAPAP,1,<NA>,Eosin 1% solution
6210,190800000AABTBT,1,<NA>,Eosin 2% solution
